In [2]:
import os
os.getcwd()

'/Users/balajirajaram'

In [3]:
os.chdir('/Users/balajirajaram/Desktop/Learning/ML/Titanic_Challenge')

In [5]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import statsmodels.api as sm
from statsmodels.nonparametric.kde import KDEUnivariate
from statsmodels.nonparametric import smoothers_lowess
from pandas import Series, DataFrame
from patsy import dmatrices
from sklearn import datasets, svm

In [6]:
df = pd.read_csv("train.csv") 

In [72]:
if len(df.Fare[ df.Fare.isnull() ]) > 0:
    median_fare = np.zeros(3)
    for f in range(0,3):                                              # loop 0 to 2
        median_fare[f] = df[ df.Pclass == f+1 ]['Fare'].dropna().median()
    for f in range(0,3):                                              # loop 0 to 2
        df.loc[ (df.Fare.isnull()) & (df.Pclass == f+1 ), 'Fare'] = median_fare[f]

In [73]:
df['FamilySize'] = df.SibSp + df.Parch

In [74]:
df['FarePerPerson']=df.Fare/ ( df.FamilySize + 1 )

In [51]:
df['Title'] = df.Name.str.extract('.*, ?(\S+\s*\S*)\. ',expand=False)
#df.Title.value_counts()

In [52]:
#replacing all titles with mr, mrs, miss, master
def replace_titles(x):
    title=x['Title']
    if title in ['Don', 'Major', 'Capt', 'Jonkheer', 'Rev', 'Col','Sir']:
        return 'Mr'
    elif title in ['the Countess', 'Mme','Lady']:
        return 'Mrs'
    elif title in ['Mlle', 'Ms']:
        return 'Miss'
    elif title =='Dr':
        if x['Sex']=='Male':
            return 'Mr'
        else:
            return 'Mrs'
    else:
        return title

In [53]:
df['Title']=df.apply(replace_titles, axis=1)
df.Title.value_counts()

Mr        531
Miss      185
Mrs       135
Master     40
Name: Title, dtype: int64

In [59]:
df['TitleNum'] = df.Title.map({ 'Mr':1, 'Mrs':2, 'Miss':3, 'Master':4 })
df['TitleNum'].value_counts()

1    531
3    185
2    135
4     40
Name: TitleNum, dtype: int64

In [60]:
df['Gender'] = df.Sex.apply(lambda x: 0 if x == 'male' else 1)
df.Gender.value_counts()

0    577
1    314
Name: Gender, dtype: int64

In [87]:
df['AgeFill'] = df.Age

In [88]:
impute_grps = df.pivot_table(values=['AgeFill'],index=['Sex','Pclass','Title'],aggfunc=np.mean)

In [89]:
for i,row in df.loc[df['AgeFill'].isnull(),:].iterrows():
    ind = tuple([row['Sex'],row['Pclass'],row['Title']])
    df.loc[i,'AgeFill'] = impute_grps.loc[ind].values[0]

In [99]:
df['AgeNorm'] = 0
df.loc[df['AgeFill'] > 18,'AgeNorm'] = 1

In [101]:
df.AgeNorm.value_counts()

1    715
0    176
Name: AgeNorm, dtype: int64

In [102]:
df['Mother'] = 0

In [103]:
def find_mom(x):
    if x.Sex == 'female' and x.Parch > 0 and x.Title == 'Mrs':
        return 1
    else:
        return 0

In [104]:
df['Mother'] = df.apply(find_mom,axis=1)

In [105]:
df.Mother.value_counts()

0    835
1     56
Name: Mother, dtype: int64

In [112]:
(df.FarePerPerson.max() - df.FarePerPerson.mean()) / df.FarePerPerson.std()
df['FareNorm'] = (df.FarePerPerson - df.FarePerPerson.mean()) / df.FarePerPerson.std()

In [114]:
df[['FarePerPerson','FareNorm']].head()

,FarePerPerson,FareNorm
0,3.62500,-0.454543
1,35.64165,0.438748
2,7.92500,-0.334569
3,26.55000,0.185083
4,8.05000,-0.331081


In [115]:
Ports = list(enumerate(np.unique(df['Embarked'].dropna().astype(str))))
# determine all values of Embarked,
Ports_dict = { name : i for i, name in Ports }  
df.Embarked = df.Embarked.dropna().map( lambda x: Ports_dict[x]).astype(int)     # Convert all Embark strings to int

# Embarked from 'C', 'Q', 'S'
# All missing Embarked -> just make them embark from most common place
if df.Embarked.isnull().values.any():
    df.loc[ df['Embarked'].isnull() , 'Embarked'] = df['Embarked'].dropna().mode().values.astype(int)

In [120]:
df.Embarked.value_counts()

2    646
0    168
1     77
Name: Embarked, dtype: int64

In [123]:
df['AgeIsNull'] = 0
df.loc[df.Age.isnull(),'AgeIsNull'] = 1

In [125]:
df = df.drop(['PassengerId','Name','Sex','Age','Ticket','Cabin','Title'],axis=1)

In [126]:
df.dtypes

Survived           int64
Pclass             int64
SibSp              int64
Parch              int64
Fare             float64
Embarked           int64
FamilySize         int64
FarePerPerson    float64
Gender             int64
TitleNum           int64
AgeFill          float64
AgeNorm            int64
Mother             int64
FareNorm         float64
AgeIsNull          int64
dtype: object